In [8]:
#Import libraries needed 
import pandas as pd 
import numpy as np 
import yfinance as yf 
import datetime as dt 
from pandas_datareader import data as pdr 

In [9]:
#activate yahoo finance workaround
yf.pdr_override()

#ask the user what stock they want to look at, stock = input
stock = input("Enter a stock ticker symbol: ")
print(stock)

Enter a stock ticker symbol: tqqq
tqqq


In [10]:
#startdate data
startyear = 2019
startmonth = 1
startday = 1

#datetime object dictating the time of when we want the system to acess the data
start = dt.datetime(startyear,startmonth,startday)

now = dt.datetime.now()

df = pdr.get_data_yahoo(stock,start,now)

print(df)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-01-02   34.779999   38.230000   34.730000   37.570000   37.543098   
2019-01-03   35.980000   36.410000   33.630001   33.930000   33.905704   
2019-01-04   35.549999   38.759998   35.169998   38.270000   38.242596   
2019-01-07   38.470001   40.150002   38.130001   39.490002   39.461723   
2019-01-08   40.689999   41.110001   38.900002   40.630001   40.600906   
...                ...         ...         ...         ...         ...   
2020-09-04  145.380005  150.300003  123.860001  141.639999  141.639999   
2020-09-08  124.949997  133.970001  121.059998  121.620003  121.620003   
2020-09-09  129.029999  135.199997  125.669998  132.070007  132.070007   
2020-09-10  136.520004  138.389999  121.739998  124.430000  124.430000   
2020-09-11  127.000000  128.190002  116.809

In [11]:
emasUsed = [3,5,8,10,12,15,30,35,40,45,50,60]
for x in emasUsed:
    ema=x
    df["Ema_"+str(ema)]=round(df.iloc[:,4].ewm(span=ema, adjust=False).mean(),2)
    
print(df.tail())

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-09-04  145.380005  150.300003  123.860001  141.639999  141.639999   
2020-09-08  124.949997  133.970001  121.059998  121.620003  121.620003   
2020-09-09  129.029999  135.199997  125.669998  132.070007  132.070007   
2020-09-10  136.520004  138.389999  121.739998  124.430000  124.430000   
2020-09-11  127.000000  128.190002  116.809998  121.709999  121.709999   

              Volume   Ema_3   Ema_5   Ema_8  Ema_10  Ema_12  Ema_15  Ema_30  \
Date                                                                           
2020-09-04  70185100  150.07  153.34  153.50  152.47  151.11  148.88  138.47   
2020-09-08  53353100  135.85  142.77  146.42  146.86  146.57  145.47  137.38   
2020-09-09  36294000  133.96  139.20  143.23  144.17  144.34  143.79  137.04   
2020-09-10  58242100  129.19  134.28  139.05  140.58  141.28  141.37  136.22   
2

In [12]:
pos=0
num=0
percentchange=[]

for i in df.index:
    cmin=min(df["Ema_3"][i],df["Ema_5"][i],df["Ema_8"][i],df["Ema_10"][i],df["Ema_12"][i],df["Ema_15"][i],)
    cmax=max(df["Ema_30"][i],df["Ema_35"][i],df["Ema_40"][i],df["Ema_45"][i],df["Ema_50"][i],df["Ema_60"][i],)
    
    close=df["Adj Close"][i]
    
    if(cmin>cmax):
        print("Red White Blue")
        if(pos==0):
            bp=close
            pos=1
            print("Buying now at "+str(bp))
            
        
    elif(cmin<cmax):
        print("Blue White Red")
        if(pos==1):
            pos=0
            sp=close
            print("Selling now at "+str(sp))
            pc=(sp/bp-1)*100
            percentchange.append(pc)
    if(num==df["Adj Close"].count()-1 and pos==1):
        pos=0
        sp=close
        print("Selling now at "+str(sp))
        pc=(sp/bp-1)*100
        percentchange.append(pc)
        
    num+=1
    
print(percentchange)
                  

Blue White Red
Blue White Red
Blue White Red
Red White Blue
Buying now at 40.60090637207031
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red Whit

In [13]:
gains=0
ng=0
losses=0
nl=0
totalR=1

for i in percentchange:
    if(i>0):
        gains+=i
        ng+=1
    else:
        losses+=i
        nl+=1
        totalR=totalR*((i/100)+1)

totalR=round((totalR-1)*100,2)

if(ng>0):
    avgGain=gains/ng
    maxR=str(max(percentchange))
else:
    avgGain=0
    maxR="undefined"
if(nl>0):
    avgLoss=losses/nl
    maxL=str(min(percentchange))
    ratio=str(-avgGain/avgLoss)
else:
    avgGain=0
    maxR="undefined"
    ratio="inf"
    
if(ng>0 or nl>0):
    battingAvg=ng/(ng+nl)
else:
    battingAvg=0
    
print()
print("Results for "+ stock +" going back to "+str(df.index[0])+", Sample size: "+str(ng+nl)+" trades")
print("EMAs used: "+str(emasUsed))
print("Batting Avg: "+ str(battingAvg))
print("Gain/loss ratio: "+ ratio)
print("Average Gain: "+ str(avgGain))
print("Average Loss: "+ str(avgLoss))
print("Max Return: "+ maxR)
print("Max Loss: "+ maxL)
print("Total return over "+str(ng+nl)+ " trades: "+ str(totalR)+"%" )
#print("Example return Simulating "+str(n)+ " trades: "+ str(nReturn)+"%" )
print()

    
    


Results for tqqq going back to 2019-01-02 00:00:00, Sample size: 5 trades
EMAs used: [3, 5, 8, 10, 12, 15, 30, 35, 40, 45, 50, 60]
Batting Avg: 0.6
Gain/loss ratio: 13.945042421459949
Average Gain: 47.41214085588009
Average Loss: -3.3999280477568004
Max Return: 58.669276354182664
Max Loss: -6.444678771463308
Total return over 5 trades: -6.78%

